<h1>template notebook</h1>

# import libraries

In [8]:
#defaul libraries
#https://docs.python.org/ja/
import os
import sys
import io

import math
import random
import pprint
import time
import datetime
import typing
import json
import glob
import requests
import warnings
import gc
from pprint import pprint
import re
import pathlib

import numpy as np #https://numpy.org/
import pandas as pd #https://pandas.pydata.org/
import sklearn #https://scikit-learn.org/stable/

import matplotlib.pyplot as plt #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
%matplotlib inline

import seaborn as sns
sns.set()

# from tqdm import tqdm #https://tqdm.github.io/

# import torch #https://pytorch.org/
# import transformers #https://huggingface.co/transformers/

# import torchvision
# import torchtext

In [9]:
print(os.curdir)
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

.
./feedback-prize-2021-eda.ipynb


# Configuration

In [10]:
class CFG():
    
    data_path="../../data/feedback-prize-2021/"
    debug=True
    seed=0

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    batch_size=1#32
    epochs=30
    learning_rate=0.1
    kFold=5

    amp=False

def set_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #tf.random.set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(CFG.seed)

print(CFG.device)
if torch.cuda.is_available():
    torch.cuda.set_device(CFG.device)
    print(torch.cuda.current_device())
    print(torch.cuda.get_device_name())

if CFG.debug:
    CFG.epochs=5

cpu


# Color

In [11]:
def color(string,fg='DEFAULT',bg='DEFAULT',fg_rgb=None,bg_rgb=None,style='END'):
    colors=['BLACK','RED','GREEN','YELLOW','BLUE','PURPLE','CYAN','WHITE','8','DEFAULT']
    styles=['END','BOLD','2','3','UNDERLINE','5','6','REVERSE','INVISIBLE','9']

    fg=f'\033[3{colors.index(fg)}m'
    bg=f'\033[4{colors.index(bg)}m'
    style=f'\033[0{styles.index(style)}m'

    if fg_rgb:fg=f"\033[38;2;{fg_rgb[0]};{fg_rgb[1]};{fg_rgb[2]}m"
    if bg_rgb:bg=f"\033[48;2;{bg_rgb[0]};{bg_rgb[1]};{bg_rgb[2]}m"

    return style+fg+bg+str(string)+'\033[0m'

In [12]:
target_columns=['target']
def feature_columns(df):
    for col in df.columns:
        if col in target_columns:df=df.drop(columns=col)
    return  df.columns
def numerical_columns(df):
    return df[feature_columns(df)].select_dtypes(include=['int64','float64']).columns
def categorical_columns(df):
    return df[feature_columns(df)].select_dtypes(exclude=['int64','float64']).columns

# read data

In [13]:
train=pd.read_csv(os.path.join(CFG.data_path,"train.csv"),index_col='id')
# test=pd.read_csv(os.path.join(CFG.data_path,"test.csv"),index_col='id')
sample_submission=pd.read_csv(os.path.join(CFG.data_path,"sample_submission.csv"),index_col='id')

# if CFG.debug:
#     #train=train[:len(train)//50]
#     train=train.sample(frac=0.1,random_state=CFG.seed)

# df=pd.concat([train,test])
# train=df[:len(train)].copy()
# test=df[len(train):].copy()

# data={
#     'train':pd.DataFrame(columns=['feature','target']),
#     'val':pd.DataFrame({'feature':val.feature,'target':val['target']}),
#     'test':pd.DataFrame(test),
# }

In [14]:
train

,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
id,,,,,,,
423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
...,...,...,...,...,...,...,...
4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...
4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...
4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838


In [30]:
train.index.unique().values[:10]

array(['423A1CA112E2', 'A8445CABFECE', '6B4F7A0165B9', 'E05C7F5C1156',
       '50B3435E475B', 'DBF7EB6A9E02', '810B70E80E1D', 'CE98789F502B',
       'A97DE0D49AEA', '48D3F4243F0F'], dtype=object)

In [16]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,indices,phase='train',transform=None):
        self.indices=indices
        self.phase=phase #train/val/test
        self.transform=transform
        
    def __len__(self):
        return len(self.indices)

    def __getitem__(self,idx):
        
        
        path=pathlib.Path(self.indices[idx])
        text = path.read_text()
        annotation=train[train.index==path.stem]
        
        
        return text,annotation
        #return {'digit':datum,'label':self.labels.loc[idx]}

In [17]:
train_texts=glob.glob(CFG.data_path+"train/*")
train_texts.sort()

if CFG.debug:
    #train=train[:len(train)//50]
    train_texts=random.sample(train_texts, len(train_texts)//10)


test_texts=glob.glob(CFG.data_path+"test/*")
test_texts.sort()


data={
    'train':Dataset(train_texts),
    'test' :Dataset(test_texts),
}

In [18]:
pip install --upgrade spacy;

  Using cached spacy-3.4.1-cp38-cp38-macosx_10_9_x86_64.whl (6.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.1/752.1 kB 8.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.15
    Uninstalling thinc-8.0.15:
      Successfully uninstalled thinc-8.0.15
  Rolling back uninstall of thinc
  Moving to /Users/toutatsu/.local/lib/python3.8/site-packages/thinc-8.0.15.dist-info/
   from /Users/toutatsu/.local/lib/python3.8/site-packages/~hinc-8.0.15.dist-info
  Moving to /Users/toutatsu/.local/lib/python3.8/site-packages/thinc/
   from /Users/toutatsu/.local/lib/python3.8/site-packages/~hinc
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/Users/toutatsu/.anyenv/envs/pyenv/versions/3.8.8/lib/python3.8/site-packages/thinc/__init__.pxd'
Consider using the `--user` option or check the permissions.

Note: you may need to restart the kernel to use updated packages.


In [19]:
import spacy
from spacy import displacy
from spacy.tokens import Span

nlp = spacy.blank("en")

## annotation visualization

reference

* [Feedback Prize EDA with displacy](https://www.kaggle.com/code/thedrcat/feedback-prize-eda-with-displacy)

In [20]:
def visualize(text_id):
    ents = []
    for i, row in train[train.index== text_id].iterrows():
        ents.append(
            {
                'start': int(row['discourse_start']), 
                'end': int(row['discourse_end']), 
                'label': row['discourse_type']
            }
        )
    
    displacy.render(
        {
            "text": pathlib.Path(os.path.join(CFG.data_path,'train',text_id+'.txt')).read_text(),
            "ents": ents,
            "title": text_id
        },
        style="ent",
        options={
            "ents": train.discourse_type.unique().tolist(),
            "colors":{
                'Lead': '#8000ff',
                'Position': '#2b7ff6',
                'Evidence': '#2adddd',
                'Claim': '#80ffb4',
                'Concluding Statement': 'd4dd80',
                'Counterclaim': '#ff8042',
                'Rebuttal': '#ff0000'
            }
        },
        manual=True,
        jupyter=True
    )

In [21]:
visualize(train.index[10])

In [22]:
visualize('149E8C278863')

submission

In [23]:
submission=pd.DataFrame(index=pd.Series(name='row_id',dtype='int'),columns=['idx','target'])


submission.idx=range(5)
submission.target=[1,0,0,1,1]

#submission.sort_index(inplace=True)

#submission.to_csv("submission.csv",index=idx)

submission.head()

,idx,target
row_id,,
0,0,1
1,1,0
2,2,0
3,3,1
4,4,1
